# Propositionalization: Robot sensor data

In this notebook, we compare getML's FastProp against well-known feature engineering libraries featuretools and tsfresh.


Summary:

- Prediction type: __Regression__
- Domain: __Robotics__
- Prediction target: __The force vector on the robot's arm__ 
- Population size: __15001__

<a target="_blank" href="https://colab.research.google.com/github/getml/getml-demo/blob/master/fastprop_benchmark/robot_prop.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Background

A common approach to feature engineering is to generate attribute-value representations from relational data by applying a fixed set of aggregations to columns of interest and perform a feature selection on the (possibly large) set of generated features afterwards. In academia, this approach is called _propositionalization._

getML's [FastProp](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#fastprop) is an implementation of this propositionalization approach that has been optimized for speed and memory efficiency. In this notebook, we want to demonstrate how – well – fast FastProp is. To this end, we will benchmark FastProp against the popular feature engineering libraries [featuretools](https://www.featuretools.com/) and [tsfresh](https://tsfresh.readthedocs.io/en/latest/). Both of these libraries use propositionalization approaches for feature engineering.

The data set has been generously provided by Erik Berger who originally collected it for his dissertation:

> Berger, E. (2018). *Behavior-Specific Proprioception Models for Robotic Force Estimation: A Machine Learning Approach.* Freiberg, Germany: Technische Universitaet Bergakademie Freiberg.

## Analysis

1. [Loading data](#1.-Loading-data)
2. [Predictive modeling](#2.-Predictive-modeling)
3. [Comparison](#3.-Comparison)

We begin by importing the libraries and setting the project.

In [1]:
%pip install -q "getml==1.5.0" "featuretools==1.31.0" "tsfresh==0.20.3"

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import sys

os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
from pathlib import Path

import numpy as np
import pandas as pd
import getml

print(f"getML API version: {getml.__version__}\n")

getML API version: 1.5.0



In [3]:
getml.engine.launch(allow_remote_ips=True, token="token")
getml.engine.set_project("robot")

getML Engine is already running.


Connected to project 'robot'.

In [4]:
# If we are in Colab, we need to fetch the utils folder from the repository
if os.getenv("COLAB_RELEASE_TAG"):
    !curl -L https://api.github.com/repos/getml/getml-demo/tarball/master | tar --wildcards --strip-components=1 -xz '*utils*'

In [5]:
parent = Path(os.getcwd()).parent.as_posix()

if parent not in sys.path:
    sys.path.append(parent)

from utils import Benchmark, FTTimeSeriesBuilder, TSFreshBuilder

### 1. Loading data

#### 1.1 Download from source


In [6]:
data_all = getml.data.DataFrame.from_csv(
    "https://static.getml.com/datasets/robotarm/robot-demo.csv", "data_all"
)

Downloading https://static.getml.com/datasets/robotarm/robot-demo.csv to 
/tmp/getml/static.getml.com/datasets/robotarm/robot-demo.csv...

In [7]:
data_all

name              3              4              5              6   ...            105            106            f_x
 role   unused_float   unused_float   unused_float   unused_float   ...   unused_float   unused_float   unused_float
    0         3.4098        -0.3274         0.9604        -3.7436   ...         47.955         47.971        -11.03 
    1         3.4098        -0.3274         0.9604        -3.7436   ...         47.955         47.971        -10.848
    2         3.4098        -0.3274         0.9604        -3.7436   ...         47.955         47.971        -10.666
    3         3.4098        -0.3274         0.9604        -3.7436   ...         47.955         47.971        -10.507
    4         3.4098        -0.3274         0.9604        -3.7436   ...         47.955         47.971        -10.413
             ...            ...            ...            ...                   ...            ...            ...   
14996         3.0837        -0.8836         1.4501        -2.2102   ...         47.94          47.94          10.84 
14997         3.0835        -0.884          1.4505        -2.2091   ...         47.94          47.94          10.857
14998         3.0833        -0.8844         1.4508        -2.208    ...         47.94          47.94          10.89 
14999         3.0831        -0.8847         1.4511        -2.2071   ...         47.94          47.94          11.29 
15000         3.0829        -0.885          1.4514        -2.2062   ...         47.94          47.955         11.69 

 name            f_y            f_z
 role   unused_float   unused_float
    0         6.9           -7.33  
    1         6.7218        -7.4427
    2         6.5436        -7.5555
    3         6.4533        -7.65  
    4         6.6267        -7.69  
             ...            ...    
14996        -1.41          16.14  
14997        -1.52          15.943 
14998        -1.74          15.55  
14999        -1.4601        15.743 
15000        -1.1801        15.937 


15001 rows x 96 columns
memory usage: 11.52 MB
type: getml.DataFrame

#### 1.2 Prepare data

The force vector consists of three component (*f_x*, *f_y* and *f_z*), meaning that we have three targets. For this comparison, we only predict the first component (*f_x*). 

Also, we want to speed things up a little, so we only use 10 columns. A previous analysis has revealed that the predictive power is mainly extracted from these 10 columns:

In [8]:
only_use = ["30", "34", "37", "38", "4", "59", "61", "7", "77", "78"]

data_all.set_role(["f_x"], getml.data.roles.target)
data_all.set_role(only_use, getml.data.roles.numerical)

This is what the data set looks like:

In [9]:
data_all

name       f_x          30          34          37   ...            104            105            106            f_y
 role    target   numerical   numerical   numerical   ...   unused_float   unused_float   unused_float   unused_float
    0   -11.03      -1.2042     -0.3274     -1.0191   ...         47.834         47.955         47.971         6.9   
    1   -10.848     -1.2042     -0.3274     -1.0191   ...         47.834         47.955         47.971         6.7218
    2   -10.666     -1.2042     -0.3274     -1.0191   ...         47.834         47.955         47.971         6.5436
    3   -10.507     -1.2042     -0.3273     -1.0191   ...         47.834         47.955         47.971         6.4533
    4   -10.413     -1.2042     -0.3274     -1.0191   ...         47.834         47.955         47.971         6.6267
         ...        ...         ...         ...                   ...            ...            ...           ...    
14996    10.84      -2.6343     -0.8836     -1.5591   ...         47.803         47.94          47.94         -1.41  
14997    10.857     -2.6331     -0.8841     -1.5596   ...         47.803         47.94          47.94         -1.52  
14998    10.89      -2.6321     -0.8844     -1.56     ...         47.803         47.94          47.94         -1.74  
14999    11.29      -2.6311     -0.8848     -1.5604   ...         47.803         47.94          47.94         -1.4601
15000    11.69      -2.6302     -0.8851     -1.5607   ...         47.803         47.94          47.955        -1.1801

 name            f_z
 role   unused_float
    0        -7.33  
    1        -7.4427
    2        -7.5555
    3        -7.65  
    4        -7.69  
             ...    
14996        16.14  
14997        15.943 
14998        15.55  
14999        15.743 
15000        15.937 


15001 rows x 96 columns
memory usage: 11.52 MB
type: getml.DataFrame

#### 1.3 Separate data into a training and testing set

We also want to separate the data set into a training and testing set. We do so by using the first 10,500 measurements for training and then using the remainder for testing.

In [10]:
split = getml.data.split.time(data_all, "rowid", test=10500)
split

,
0,train
1,train
2,train
3,train
4,train
,...


In [11]:
time_series = getml.data.TimeSeries(
    population=data_all,
    split=split,
    time_stamps="rowid",
    lagged_targets=False,
    memory=30,
)

time_series

data model

  population:
    columns:
    - 30: numerical
    - 34: numerical
    - 37: numerical
    - 38: numerical
    - 4: numerical
    - ...

    joins:
    - right: 'data_all'
      time_stamps: (population.rowid, data_all.rowid)
      relationship: 'many-to-many'
      memory: 30
      lagged_targets: False

  data_all:
    columns:
    - 30: numerical
    - 34: numerical
    - 37: numerical
    - 38: numerical
    - 4: numerical
    - ...


container

  population
      subset   name       rows      type
  0   test     data_all   unknown   View
  1   train    data_all   unknown   View

  peripheral
      name        rows   type
  0   data_all   15001   View

### 2. Predictive modeling

#### 2.1 Propositionalization with getML's FastProp

In [12]:
fast_prop = getml.feature_learning.FastProp(
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
)

In [13]:
pipe_fp_fl = getml.pipeline.Pipeline(
    data_model=time_series.data_model,
    feature_learners=[fast_prop],
    tags=["feature learning", "fastprop"],
)

In [14]:
pipe_fp_fl.check(time_series.train)

Checking data model...

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Checking... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


OK.

In [15]:
benchmark = Benchmark()

In [16]:
with benchmark("fastprop"):
    pipe_fp_fl.fit(time_series.train)
    fastprop_train = pipe_fp_fl.transform(time_series.train, df_name="fastprop_train")

Checking data model...

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


OK.

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  FastProp: Trying 134 features... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


Trained pipeline.

Time taken: 0:00:00.032192.

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Preprocessing... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  FastProp: Building features... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


In [17]:
fastprop_test = pipe_fp_fl.transform(time_series.test, df_name="fastprop_test")

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Preprocessing... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  FastProp: Building features... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


In [18]:
predictor = getml.predictors.XGBoostRegressor()

pipe_fp_pr = getml.pipeline.Pipeline(
    tags=["prediction", "fastprop"], predictors=[predictor]
)

In [19]:
pipe_fp_pr.check(fastprop_train)

Checking data model...

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Checking... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


The pipeline check generated 0 issues labeled INFO and 5 issues labeled WARNING.

,type,label,message
0,WARNING,COLUMN SHOULD BE UNUSED,All non-NULL entries in column 'feature_1_121' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').
1,WARNING,COLUMN SHOULD BE UNUSED,All non-NULL entries in column 'feature_1_126' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').
2,WARNING,COLUMN SHOULD BE UNUSED,All non-NULL entries in column 'feature_1_127' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').
3,WARNING,COLUMN SHOULD BE UNUSED,All non-NULL entries in column 'feature_1_128' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').
4,WARNING,COLUMN SHOULD BE UNUSED,All non-NULL entries in column 'feature_1_132' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').


In [20]:
pipe_fp_pr.fit(fastprop_train)

Checking data model...

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


The pipeline check generated 0 issues labeled INFO and 5 issues labeled WARNING.

To see the issues in full, run .check() on the pipeline.

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  XGBoost: Training as predictor... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:04


Trained pipeline.

Time taken: 0:00:04.112529.



Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['prediction', 'fastprop'])

In [21]:
pipe_fp_pr.score(fastprop_test)

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Preprocessing... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


,date time,set used,target,mae,rmse,rsquared
0,2024-09-13 14:16:39,fastprop_train,f_x,0.4383,0.5764,0.9963
1,2024-09-13 14:16:39,fastprop_test,f_x,0.5516,0.7237,0.9951


#### 2.2 Propositionalization with featuretools

In [22]:
data_train = time_series.train.population.to_df("data_train")
data_test = time_series.test.population.to_df("data_test")

In [23]:
dfs_pandas = {}

for df in [data_train, data_test, data_all]:
    dfs_pandas[df.name] = df.to_pandas()
    delete_columns = [
        col for col in dfs_pandas[df.name].columns if col not in only_use + ["f_x"]
    ]
    for col in delete_columns:
        del dfs_pandas[df.name][col]
    dfs_pandas[df.name]["id"] = 1
    dfs_pandas[df.name]["ds"] = pd.to_datetime(
        np.arange(0, dfs_pandas[df.name].shape[0]), unit="s"
    )

In [24]:
dfs_pandas["data_train"]

,30,34,37,38,4,59,61,7,77,78,f_x,id,ds
0,-1.2042,-0.32739,-1.0191,-6.0205,-0.32737,0.082791,0.78597,-1.0191,0.082782,-1.4094,-11.0300,1,1970-01-01 00:00:00
1,-1.2042,-0.32739,-1.0191,-6.0205,-0.32737,0.082800,0.78592,-1.0191,0.082782,-1.4094,-10.8480,1,1970-01-01 00:00:01
2,-1.2042,-0.32737,-1.0191,-6.0205,-0.32737,0.082786,0.78594,-1.0191,0.082782,-1.4094,-10.6660,1,1970-01-01 00:00:02
3,-1.2042,-0.32734,-1.0191,-6.0205,-0.32737,0.082755,0.78599,-1.0191,0.082782,-1.4094,-10.5070,1,1970-01-01 00:00:03
4,-1.2042,-0.32736,-1.0191,-6.0205,-0.32737,0.082782,0.78597,-1.0191,0.082782,-1.4094,-10.4130,1,1970-01-01 00:00:04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10495,-1.1446,-0.37311,-1.0486,-5.9532,-0.37326,0.087343,0.90793,-1.0488,0.087468,-1.4162,-9.7673,1,1970-01-01 02:54:55
10496,-1.1349,-0.37103,-1.0472,-5.9564,-0.37108,0.087241,0.90199,-1.0474,0.087274,-1.4160,-9.9200,1,1970-01-01 02:54:56
10497,-1.1255,-0.36889,-1.0458,-5.9596,-0.36896,0.087055,0.89618,-1.0460,0.087082,-1.4158,-9.7743,1,1970-01-01 02:54:57
10498,-1.1163,-0.36680,-1.0444,-5.9627,-0.36689,0.086907,0.89034,-1.0447,0.086893,-1.4155,-8.6109,1,1970-01-01 02:54:58


In [25]:
ft_builder = FTTimeSeriesBuilder(
    num_features=200,
    horizon=pd.Timedelta(seconds=0),
    memory=pd.Timedelta(seconds=15),
    column_id="id",
    time_stamp="ds",
    target="f_x",
)

In [26]:
with benchmark("featuretools"):
    featuretools_train = ft_builder.fit(dfs_pandas["data_train"])

featuretools_test = ft_builder.transform(dfs_pandas["data_test"])

featuretools: Trying features...
Selecting the best out of 442 features...
Time taken: 0h:14m:21.609569



In [27]:
featuretools_train

,MIN(peripheral.37),MIN(peripheral.7),FIRST(peripheral.37),FIRST(peripheral.7),MEDIAN(peripheral.37),MEDIAN(peripheral.7),MEAN(peripheral.37),MEAN(peripheral.7),SUM(peripheral.37),SUM(peripheral.7),...,NUM_ZERO_CROSSINGS(peripheral.77),NUM_ZERO_CROSSINGS(peripheral.59),IS_MONOTONICALLY_DECREASING(peripheral.77),"TIME_SINCE_LAST_MAX(peripheral.ds, 4)","TIME_SINCE_LAST_MAX(peripheral.ds, 30)","TIME_SINCE_LAST_MAX(peripheral.ds, 77)","TIME_SINCE_LAST_MAX(peripheral.ds, 7)",f_x,id,ds
_featuretools_index,,,,,,,,,,,,,,,,,,,,,
0,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.019100,-1.019100,-1.0191,-1.0191,...,0,0,True,1.726237e+09,1.726237e+09,1.726237e+09,1.726237e+09,-11.0300,1,1970-01-01 00:00:00
1,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.019100,-1.019100,-2.0382,-2.0382,...,0,0,True,1.726237e+09,1.726237e+09,1.726237e+09,1.726237e+09,-10.8480,1,1970-01-01 00:00:01
2,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.019100,-1.019100,-3.0573,-3.0573,...,0,0,True,1.726237e+09,1.726237e+09,1.726237e+09,1.726237e+09,-10.6660,1,1970-01-01 00:00:02
3,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.019100,-1.019100,-4.0764,-4.0764,...,0,0,True,1.726237e+09,1.726237e+09,1.726237e+09,1.726237e+09,-10.5070,1,1970-01-01 00:00:03
4,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.019100,-1.019100,-5.0955,-5.0955,...,0,0,True,1.726237e+09,1.726237e+09,1.726237e+09,1.726237e+09,-10.4130,1,1970-01-01 00:00:04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10495,-1.0716,-1.0721,-1.0716,-1.0721,-1.0596,-1.0596,-1.059793,-1.059927,-15.8969,-15.8989,...,0,0,True,1.726227e+09,1.726227e+09,1.726227e+09,1.726227e+09,-9.7673,1,1970-01-01 02:54:55
10496,-1.0698,-1.0702,-1.0698,-1.0702,-1.0580,-1.0580,-1.058167,-1.058280,-15.8725,-15.8742,...,0,0,True,1.726227e+09,1.726227e+09,1.726227e+09,1.726227e+09,-9.9200,1,1970-01-01 02:54:56
10497,-1.0681,-1.0684,-1.0681,-1.0684,-1.0565,-1.0563,-1.056567,-1.056667,-15.8485,-15.8500,...,0,0,True,1.726227e+09,1.726227e+09,1.726227e+09,1.726227e+09,-9.7743,1,1970-01-01 02:54:57


In [28]:
roles = {
    getml.data.roles.target: ["f_x"],
    getml.data.roles.join_key: ["id"],
    getml.data.roles.time_stamp: ["ds"],
}

df_featuretools_train = getml.data.DataFrame.from_pandas(
    featuretools_train, name="featuretools_train", roles=roles
)

df_featuretools_test = getml.data.DataFrame.from_pandas(
    featuretools_test, name="featuretools_test", roles=roles
)

In [29]:
df_featuretools_train.set_role(
    df_featuretools_train.roles.unused, getml.data.roles.numerical
)

df_featuretools_test.set_role(
    df_featuretools_test.roles.unused, getml.data.roles.numerical
)

In [30]:
predictor = getml.predictors.XGBoostRegressor()

pipe_ft_pr = getml.pipeline.Pipeline(
    tags=["prediction", "featuretools"], predictors=[predictor]
)

pipe_ft_pr

Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['prediction', 'featuretools'])

In [31]:
pipe_ft_pr.fit(df_featuretools_train)

Checking data model...

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Checking... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


OK.

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  XGBoost: Training as predictor... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:04


Trained pipeline.

Time taken: 0:00:04.476476.



Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['prediction', 'featuretools'])

In [32]:
pipe_ft_pr.score(df_featuretools_test)

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Preprocessing... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


,date time,set used,target,mae,rmse,rsquared
0,2024-09-13 14:37:28,featuretools_train,f_x,0.4396,0.584,0.9962
1,2024-09-13 14:37:28,featuretools_test,f_x,0.5828,0.7595,0.9948


#### 2.3 Propositionalization with tsfresh

In [33]:
tsfresh_builder = TSFreshBuilder(
    num_features=200,
    memory=15,
    column_id="id",
    time_stamp="ds",
    target="f_x",
)

In [34]:
with benchmark("tsfresh"):
    tsfresh_train = tsfresh_builder.fit(dfs_pandas["data_train"])

tsfresh_test = tsfresh_builder.transform(dfs_pandas["data_test"])

Feature Extraction: 100%|██████████| 40/40 [00:11<00:00,  3.64it/s]


Selecting the best out of 130 features...
Time taken: 0h:0m:26.638195



Feature Extraction: 100%|██████████| 40/40 [00:04<00:00,  9.93it/s]


In [35]:
roles = {
    getml.data.roles.target: ["f_x"],
    getml.data.roles.join_key: ["id"],
    getml.data.roles.time_stamp: ["ds"],
}

df_tsfresh_train = getml.data.DataFrame.from_pandas(
    tsfresh_train, name="tsfresh_train", roles=roles
)

df_tsfresh_test = getml.data.DataFrame.from_pandas(
    tsfresh_test, name="tsfresh_test", roles=roles
)

In [36]:
df_tsfresh_train.set_role(df_tsfresh_train.roles.unused, getml.data.roles.numerical)

df_tsfresh_test.set_role(df_tsfresh_test.roles.unused, getml.data.roles.numerical)

In [37]:
pipe_tsf_pr = getml.pipeline.Pipeline(
    tags=["predicition", "tsfresh"], predictors=[predictor]
)

pipe_tsf_pr

Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['predicition', 'tsfresh'])

In [38]:
pipe_tsf_pr.check(df_tsfresh_train)

Checking data model...

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Checking... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


OK.

In [39]:
pipe_tsf_pr.fit(df_tsfresh_train)

Checking data model...

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


OK.

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  XGBoost: Training as predictor... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:03


Trained pipeline.

Time taken: 0:00:03.942466.



Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['predicition', 'tsfresh'])

In [40]:
pipe_tsf_pr.score(df_tsfresh_test)

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Preprocessing... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


,date time,set used,target,mae,rmse,rsquared
0,2024-09-13 14:38:12,tsfresh_train,f_x,0.4916,0.6636,0.9951
1,2024-09-13 14:38:12,tsfresh_test,f_x,0.5986,0.7906,0.9938


### 3. Comparison

In [41]:
num_features = dict(
    fastprop=134,
    featuretools=158,
    tsfresh=120,
)

runtime_per_feature = [
    benchmark.runtimes["fastprop"] / num_features["fastprop"],
    benchmark.runtimes["featuretools"] / num_features["featuretools"],
    benchmark.runtimes["tsfresh"] / num_features["tsfresh"],
]

features_per_second = [1.0 / r.total_seconds() for r in runtime_per_feature]

normalized_runtime_per_feature = [
    r / runtime_per_feature[0] for r in runtime_per_feature
]

comparison = pd.DataFrame(
    dict(
        runtime=[
            benchmark.runtimes["fastprop"],
            benchmark.runtimes["featuretools"],
            benchmark.runtimes["tsfresh"],
        ],
        num_features=num_features.values(),
        features_per_second=features_per_second,
        normalized_runtime=[
            1,
            benchmark.runtimes["featuretools"] / benchmark.runtimes["fastprop"],
            benchmark.runtimes["tsfresh"] / benchmark.runtimes["fastprop"],
        ],
        normalized_runtime_per_feature=normalized_runtime_per_feature,
        rsquared=[pipe_fp_pr.rsquared, pipe_ft_pr.rsquared, pipe_tsf_pr.rsquared],
        rmse=[pipe_fp_pr.rmse, pipe_ft_pr.rmse, pipe_tsf_pr.rmse],
        mae=[pipe_fp_pr.mae, pipe_ft_pr.mae, pipe_tsf_pr.mae],
    )
)

comparison.index = ["getML: FastProp", "featuretools", "tsfresh"]

In [42]:
comparison

,runtime,num_features,features_per_second,normalized_runtime,normalized_runtime_per_feature,rsquared,rmse,mae
getML: FastProp,0 days 00:00:00.398347,134,336.360579,1.000000,1.000000,0.995058,0.723716,0.551608
featuretools,0 days 00:14:21.611109,158,0.183377,2162.966230,1834.253280,0.994784,0.759460,0.582831
tsfresh,0 days 00:00:26.638362,120,4.504789,66.872255,74.667339,0.993836,0.790602,0.598600


In [43]:
comparison.to_csv("comparisons/robot.csv")